Code Runtime 指的是你的代码运行时所处的计算环境. 例如是在 laptop 上? 还是在 CI 的环境中? 还是在 AWS 计算环境中 (例如 Lambda)? 我们的代码逻辑需要知道自己当前所在的 Runtime 才能使用不同的逻辑分支来进行工作. 例如你要读一个数据库密码, 在 laptop 上你可能直接去本地文件读了, 在 CI 的环境中你可能会去 Environment Variable 中读, 在 AWS 的计算环境中你可能会去 Parameter Store 读. 所以你的代码中需要有一个模块能检测到当前的 Runtime, 然后用 if else 或者 match 来决定具体的操作.

``aws_ops_alpha`` 有一个模块 runtime 已经实现了很多自动检测当前 runtime 的逻辑. 你只需要 import 之后使用即可. 当然, 如果你的 runtime 检测逻辑和默认的不同, 例如默认是用 ``BUILD_TAG`` 和 ``EXECUTOR_NUMBER`` 两个环境变量检测是不是在 Jenkins 中的. 如果你的检测方式不同, 那么你可以 override 这个类, 自己实现对应的逻辑, 并且实例化一个 ``runtime = Runtime()`` 对象即可.

```python
class Runtime:
    ...
    @cached_property
    def is_jenkins(self) -> bool:
        """
        Reference:

        - https://www.jenkins.io/doc/book/pipeline/jenkinsfile/#using-environment-variables
        """
        if _check_user_env_var(RunTimeEnum.jenkins.value):  # pragma: no cover
            return True
        return "BUILD_TAG" in os.environ and "EXECUTOR_NUMBER" in os.environ
```

下面的例子展示了无需修改, 直接 import 的使用方法.

In [2]:
import aws_ops_alpha.api as aws_ops_alpha

runtime = aws_ops_alpha.runtime

In [4]:
runtime.current_runtime

'local'

In [5]:
runtime.current_runtime_group

'local'

In [6]:
runtime.is_local

True

In [8]:
runtime.is_github_action

False

In [9]:
runtime.is_aws_lambda

False

下面的例子展示了自定义 Runtime 检测逻辑的使用方法.

In [10]:
import os
import aws_ops_alpha.api as aws_ops_alpha

class Runtime(aws_ops_alpha.Runtime):
    @property
    def is_local(self) -> bool:
        return "IS_LOCAL" in os.environ

runtime = Runtime()
runtime.is_local

False